In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [2]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)


In [4]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')

mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("sharpsalt").project("multilane-capstone-mcfeq")
version = project.version(1)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to multilane-capstone-1 in yolov11:: 100%|██████████| 1710/1710 [00:00<00:00, 7702.11it/s]


In [13]:
dataset.location

'/content/multilane-capstone-1'

In [14]:
%cd {HOME}

/content


In [15]:
!yolo task=segment mode=train model=yolo11m-seg.pt data={dataset.location}/data.yaml epochs=90 imgsz=640 plots=True

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/multilane-capstone-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=90, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, pl

# Task
Define the Neural Collaborative Filtering (NCF) model architecture using PyTorch's `nn.Module` with embedding layers for users and artists, and a multi-layer perceptron (MLP) for interaction modeling.

## Define NCF Model

### Subtask:
Define the Neural Collaborative Filtering (NCF) model architecture using PyTorch's `nn.Module` with embedding layers for users and artists, and a multi-layer perceptron (MLP) for interaction modeling.


**Reasoning**:
I will define the NCF model architecture as requested in the instructions, including embedding layers for users and artists, and a multi-layer perceptron with a sigmoid activation for the final prediction.



In [ ]:
class NCF(nn.Module):
    def __init__(self, num_users, num_artists, embed_dim, mlp_layers):
        super().__init__()
        self.user_embedding = nn.Embedding(num_users, embed_dim)
        self.artist_embedding = nn.Embedding(num_artists, embed_dim)

        mlp_modules = []
        input_size = 2 * embed_dim
        for i, layer_size in enumerate(mlp_layers):
            mlp_modules.append(nn.Linear(input_size, layer_size))
            mlp_modules.append(nn.ReLU())
            input_size = layer_size

        mlp_modules.append(nn.Linear(input_size, 1))
        mlp_modules.append(nn.Sigmoid())
        self.mlp = nn.Sequential(*mlp_modules)

    def forward(self, user_ids, artist_ids):
        user_embedded = self.user_embedding(user_ids)
        artist_embedded = self.artist_embedding(artist_ids)

        # Concatenate user and artist embeddings
        concat_features = torch.cat([user_embedded, artist_embedded], dim=-1)

        return self.mlp(concat_features)

**Reasoning**:
The NCF model architecture has been defined. Now, I will instantiate the NCF model, define its loss function and optimizer, and then implement a training loop for it using the training data (`df_train`). This will make the `ncf_model` available for use in the `recommend` function, resolving the `NameError`.



In [ ]:
ncf_embed_dim = 64
ncf_mlp_layers = [128, 64, 32] # Example MLP layer sizes
ncf_model = NCF(num_users, num_artists, ncf_embed_dim, ncf_mlp_layers)
ncf_optimizer = optim.Adam(ncf_model.parameters(), lr=0.001)
ncf_loss_fn = nn.BCELoss()

# Prepare data for NCF training
users_tensor = torch.tensor(df_train['userId'].values, dtype=torch.long)
artists_tensor = torch.tensor(df_train['artistName'].values, dtype=torch.long)
listens_scaled = df_train['listens'].apply(lambda x: 1.0 if x > 0 else 0.0) # Convert listens to binary feedback (1 if listened, 0 otherwise)
ratings_tensor = torch.tensor(listens_scaled.values, dtype=torch.float)

def train_ncf(epochs=10, batch_size=256):
    dataset = torch.utils.data.TensorDataset(users_tensor, artists_tensor, ratings_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    ncf_model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_users, batch_artists, batch_ratings in dataloader:
            ncf_optimizer.zero_grad()
            predictions = ncf_model(batch_users, batch_artists).squeeze()
            loss = ncf_loss_fn(predictions, batch_ratings)
            loss.backward()
            ncf_optimizer.step()
            total_loss += loss.item()
        print(f"NCF Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

train_ncf()


NCF Epoch 1/10, Loss: 0.1486
NCF Epoch 2/10, Loss: 0.0002
NCF Epoch 3/10, Loss: 0.0001
NCF Epoch 4/10, Loss: 0.0001
NCF Epoch 5/10, Loss: 0.0000
NCF Epoch 6/10, Loss: 0.0000
NCF Epoch 7/10, Loss: 0.0000
NCF Epoch 8/10, Loss: 0.0000
NCF Epoch 9/10, Loss: 0.0000
NCF Epoch 10/10, Loss: 0.0000


**Reasoning**:
The NCF model has been defined and trained, resolving the `NameError`. Now, I will re-run the evaluation step to calculate Precision@K and Recall@K for the ensemble model, which now includes the NCF component.



In [ ]:
prec, rec = evaluate(k=10)
print(f"Average Precision@10: {prec:.4f}, Recall@10: {rec:.4f}")

Average Precision@10: 0.0049, Recall@10: 0.0059


## Final Task

### Subtask:
Confirm that the `NameError` is resolved and the NCF model is properly integrated into the recommendation system, and present the final evaluation results (Precision@10, Recall@10).


## Summary:

### Data Analysis Key Findings
*   The Neural Collaborative Filtering (NCF) model architecture was successfully defined using `nn.Module`, incorporating embedding layers for users and artists, and a multi-layer perceptron (MLP) with ReLU activations and a final Sigmoid output.
*   The NCF model was trained for 10 epochs using the Adam optimizer and BCELoss, with the training loss consistently decreasing to near zero, confirming successful model training.
*   After integrating the trained NCF model into the recommendation system, the ensemble evaluation yielded an average Precision@10 of 0.0049 and an average Recall@10 of 0.0059.

### Insights or Next Steps
*   The current evaluation metrics (Precision@10: 0.0049, Recall@10: 0.0059) are low, indicating that the recommendation system, even with the NCF integration, has significant room for improvement in recommending relevant items.
*   Further optimization efforts should focus on hyperparameter tuning for the NCF model, exploring alternative architectures, or enhancing the ensemble strategy to boost recommendation accuracy and relevance.
